In [68]:
import os
import numpy as np
import cv2
from scipy.linalg import qr

training_data = "D:/Projects/Exercise number 4/train"
test_data = "D:/Projects/Exercise number 4/test"

# تابعی برای محاسبه تصویر میانگین یک مجموعه از تصاویر آموزشی
def calculate_average(item, train_data):
    weights = [0.2989, 0.5870, 0.1140]
    number_images = []
    
    for file in os.listdir(os.path.join(train_data, item)):
        color_image = cv2.imread(os.path.join(train_data, item, file))
        color_image = color_image.astype(np.float16)
        gray_image = np.zeros((color_image.shape[0], color_image.shape[1]))
        
        for i in range(color_image.shape[2]):
            gray_image += (color_image[:, :, i] * weights[i])
        
        gray_image = gray_image.astype(np.uint8)
        number_images.append(gray_image.reshape(-1, 1))
        
    return np.mean(number_images, axis=0)

# حساب کردن تصاویر میانگین برای هر رقم
average_images = {}
for item in range(10):
    average_images[item] = calculate_average(str(item), training_data)
    
# محاسبه تجزیه کاهش یافته
def qr_decomposition_reduce_dimension(A):
    Q, R = qr(A)
    m, n = R.shape
    Q2 = Q[:, n:]
    return Q2

qr_models = {}
for digit in range(10):
    average_digit_image = average_images[digit]
    qr_model = qr_decomposition_reduce_dimension(average_digit_image)
    qr_models[digit] = qr_model

# def QR_Decomposition(matrix):
#     n = matrix.shape[1]
#     R = matrix
#     Q = np.eye(n)
    
#     for i in range(0, n-1):
#         X = R[i: , i]
#         if X[0] >= 0 :
#             V = X + np.sqrt(np.sum(np.square(abs(X)))) * np.eye(X.shape[0], 1)
#         else:
#             V = X - np.sqrt(np.sum(np.square(abs(X)))) * np.eye(X.shape[0], 1)
#         P = np.eye(X.shape[0]) - 2/float(np.inner(V.flatten(), V.flatten()))* np.dot(V, V.T)
#         H = np.block([[np.eye(i), np.zeros((i, n-i))],
#                        [np.zeros((n-i, i)), P]])
#         R = np.dot(H, R)
#         Q = np.dot(Q, H)
#     m, n = R.shape
#     Q2 = Q[:, n:]
#     return Q2


# خواندن داده‌های تست
test_images = []
test_labels = []

for digit in range(10):
    digit_folder = os.path.join(test_data, str(digit))
    
    for file in os.listdir(digit_folder):
        image = cv2.imread(os.path.join(digit_folder, file), cv2.IMREAD_GRAYSCALE)
        image_vector = image.reshape(-1, 1)
        
        test_images.append(image_vector)
        test_labels.append(digit)

# تبدیل لیست به آرایه
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# محاسبه دقت با استفاده از مدل‌های کاهش یافته
correct = 0
total = len(test_images)

for i in range(total):
    test_image = test_images[i]
    distances = []

    for digit in range(10):
        qr_model = qr_models[digit]
        reduced_test_image = np.dot(qr_model.T , test_image.reshape(-1, 1))
        distance = np.linalg.norm(reduced_test_image - qr_model.T @ qr_model @ reduced_test_image)
        distances.append(distance)

    predicted_label = np.argmin(distances)

    if predicted_label == test_labels[i]:
        correct += 1

accuracy = (correct / total) * 100
print(f"Accuracy is: {accuracy:.2f}%")


Accuracy is: 67.00%
